# Using GeMMM

(*This notebook can also be found in the [documentation](https://ukhsa-collaboration.github.io/gemmm/notebooks/example.html)*)

In this notebook we demonstrate how GeMMM can be used to sample origin-destination matrices for a specific set of MSOAs.

A table of available MSOAs is provided in the `tables` module. This includes additional information such as the Local Authority District (LAD), region and country that may be relevant when journey numbers are required for a larger area. For MSOAs in Wales and Scotland, the region and LAD columns are equivalent.

In [2]:
from gemmm.tables import gb_msoas
gb_msoas.head()

,msoa,msoa_name,lad,lad_name,region,region_name,country
0,E02000001,City of London 001,E09000001,City of London,E12000007,London,England
1,E02000002,Barking and Dagenham 001,E09000002,Barking and Dagenham,E12000007,London,England
2,E02000003,Barking and Dagenham 002,E09000002,Barking and Dagenham,E12000007,London,England
3,E02000004,Barking and Dagenham 003,E09000002,Barking and Dagenham,E12000007,London,England
4,E02000005,Barking and Dagenham 004,E09000002,Barking and Dagenham,E12000007,London,England


<br>
Suppose that we are interested in generating journey numbers for MSOAs located within the Cambridge LAD.

We can pull out the codes of MSOAs that satisfy this condition from the previous table.

In [3]:
LAD_NAME = 'Cambridge'
msoas = gb_msoas.query('lad_name==@LAD_NAME').msoa.to_numpy()

To sample journey numbers, we first provide the `OriginDestination` class with this array of MSOAs and a day type, either weekday or weekend. 

When running this for the first time, GeMMM will download [data files](https://github.com/ukhsa-collaboration/gemmm/tree/main/model_data/) required by the Fourier series and radiation models for each day type. These are cached for future use. 

In [4]:
from gemmm import OriginDestination
sampler = OriginDestination(msoas=msoas, day_type='weekday')

With this, we can now sample journey numbers for a single hour of the day or list of hours. Here, `n_realizations` specifies the number of realizations that will be generated for each hour. 

In [5]:
sampler.generate_sample(hours=[8, 12, 16], n_realizations=5)

Samples are stored as a dictionary in `X.samples` with keys given by the tuple *(x, y)*, where *x* is the hour and *y* is the realization number. Note that the numbering of realizations begins with zero.

Since the origin-destination matrices contain a high proportion of zeros when considering MSOAs over a large area, the generated matrices are stored in sparse matrix format. Specifically, the COOrdinate format is used, where the row attribute contains the indices of the start MSOAs, the col attribute contains the indices of the end MSOA, and the data attribute contains the number of journeys. The indices of the MSOAs refer to their position in the list originally provided to the `OriginDestination` class.

To present the output in a more readable format, the matrices can be converted to pandas DataFrames.

In [6]:
sampler.to_pandas(hour=8, realization=0, wide=True)

journeys                                                    \
end_msoa   E02003719 E02003720 E02003721 E02003722 E02003723 E02003724   
start_msoa                                                               
E02003719      233.0       2.0       3.0       3.0       3.0       8.0   
E02003720        1.0     269.0      20.0       2.0       9.0      14.0   
E02003721        2.0       4.0     375.0       4.0       6.0       8.0   
E02003722        2.0       2.0       5.0     372.0       1.0       4.0   
E02003723        3.0       1.0       5.0       2.0     429.0       5.0   
E02003724       11.0       8.0       9.0       3.0       7.0     562.0   
E02003725       19.0      10.0      20.0       8.0      15.0      25.0   
E02003726        2.0       0.0       6.0       0.0       3.0       5.0   
E02003727        1.0       1.0       3.0       1.0      10.0       2.0   
E02003728        8.0      13.0      19.0      11.0      17.0      20.0   
E02003729       11.0       8.0      18.0      14.0      11.0       0.0   
E02003730        9.0      15.0      27.0      13.0      19.0      24.0   
E02003731       11.0       8.0       9.0       2.0       8.0      10.0   

                                                                        \
end_msoa   E02003725 E02003726 E02003727 E02003728 E02003729 E02003730   
start_msoa                                                               
E02003719       39.0       9.0       5.0      16.0       8.0      41.0   
E02003720       44.0       5.0       6.0      27.0       9.0      43.0   
E02003721       47.0       4.0       1.0      14.0       7.0      47.0   
E02003722       14.0       6.0       3.0      14.0       7.0      23.0   
E02003723       19.0       5.0       2.0       9.0       5.0      33.0   
E02003724       75.0       9.0       4.0      15.0      16.0      32.0   
E02003725      984.0      12.0      11.0      30.0      17.0      71.0   
E02003726        9.0     348.0       1.0       1.0      10.0      11.0   
E02003727       16.0       3.0     371.0       8.0      11.0      17.0   
E02003728       48.0       4.0       4.0     383.0      19.0      25.0   
E02003729       51.0       3.0       5.0       9.0     353.0      42.0   
E02003730      105.0      11.0      21.0      30.0      22.0     715.0   
E02003731       40.0       7.0       2.0      10.0       7.0      18.0   

                      
end_msoa   E02003731  
start_msoa            
E02003719       16.0  
E02003720       30.0  
E02003721       22.0  
E02003722       24.0  
E02003723       21.0  
E02003724       35.0  
E02003725       30.0  
E02003726        8.0  
E02003727        5.0  
E02003728       23.0  
E02003729        4.0  
E02003730       22.0  
E02003731      495.0

<br>
Sometimes it may be useful to know average journey numbers rather than sampled values. These averages can be extracted from the Fourier series and radiation model data for a specific hour.

In [7]:
import numpy as np

hour = 16
fourier_mean = np.zeros((len(sampler.msoas),)*2)
fourier_mean[sampler.fourier_data.row, sampler.fourier_data.col] = sampler.fourier_data.mean[hour]

radiation_mean = sampler.radiation_data.theta[hour] * sampler.radiation_data.mean

model_mean = fourier_mean + radiation_mean

## Saving and loading samples
In some instances, it is convenient to save the generated matrices for use later. In GeMMM, this can be achieved using the `save_sample` argument when generating the samples. If True, the samples will be saved in the current directory, otherwise the path to a specific directory must be provided. The filename is automatically set using a timestamp and the day type.

In [8]:
save_file = sampler.generate_sample(hours=[8, 12, 16], n_realizations=5, save_sample=True)

Saving samples to weekday_samples_2025-04-29--16-02-29.nc


To load the samples, we again use the `OriginDestination` class, but this time provide the full file path.

In [9]:
loader = OriginDestination(file=save_file)

Available hours: 8, 12, 16
Number of realizations: 5


We can then load a matrix for one of the available hours. If a realization is not specified, one will be selected at random.

In [10]:
loader.load_sample(hour=16, realization=0, wide=True)

journeys                                                    \
end_msoa   E02003719 E02003720 E02003721 E02003722 E02003723 E02003724   
start_msoa                                                               
E02003719      221.0      13.0       5.0       6.0       1.0       4.0   
E02003720        5.0     255.0      11.0       6.0       2.0      20.0   
E02003721        2.0      10.0     295.0      10.0       7.0       9.0   
E02003722        4.0       1.0       4.0     257.0       2.0       7.0   
E02003723        9.0      10.0      11.0       4.0     412.0       9.0   
E02003724        8.0      16.0       5.0      10.0       6.0     348.0   
E02003725       34.0      55.0      32.0      20.0      28.0      67.0   
E02003726        7.0      13.0       8.0       2.0       6.0       4.0   
E02003727        6.0       6.0       8.0       3.0       2.0       9.0   
E02003728       21.0      27.0      13.0      12.0      11.0      23.0   
E02003729        6.0      13.0       7.0       2.0       2.0      11.0   
E02003730       25.0      37.0      37.0      11.0      30.0      36.0   
E02003731       19.0      21.0      16.0      15.0      12.0      39.0   

                                                                        \
end_msoa   E02003725 E02003726 E02003727 E02003728 E02003729 E02003730   
start_msoa                                                               
E02003719       28.0       7.0       6.0      12.0      14.0      18.0   
E02003720       21.0      10.0       4.0      14.0       7.0      15.0   
E02003721       34.0       5.0       5.0      10.0      16.0      20.0   
E02003722       18.0       4.0       2.0       6.0       5.0      10.0   
E02003723       29.0       4.0       3.0       9.0       7.0      26.0   
E02003724       31.0       3.0       6.0      16.0       9.0      22.0   
E02003725     1161.0      48.0      21.0      74.0      72.0     164.0   
E02003726       33.0     292.0       5.0       8.0       3.0      27.0   
E02003727       16.0       7.0     339.0       6.0       5.0      17.0   
E02003728       49.0      14.0       5.0     428.0       8.0      35.0   
E02003729       27.0       2.0       3.0       9.0     292.0      24.0   
E02003730      113.0      25.0      16.0      37.0      26.0     692.0   
E02003731       53.0      12.0       8.0      18.0      14.0      35.0   

                      
end_msoa   E02003731  
start_msoa            
E02003719        2.0  
E02003720       15.0  
E02003721       11.0  
E02003722        6.0  
E02003723        7.0  
E02003724        9.0  
E02003725       43.0  
E02003726        6.0  
E02003727        9.0  
E02003728       23.0  
E02003729        5.0  
E02003730       12.0  
E02003731      449.0